# Sources
https://www.kaggle.com/rounakbanik/movie-recommender-systems

## Simple Recommender

The Simple Recommender offers generalized recommnendations to every user based on movie popularity and (sometimes) genre. The basic idea behind this recommender is that movies that are more popular and more critically acclaimed will have a higher probability of being liked by the average audience. This model does not give personalized recommendations based on the user. 

The implementation of this model is extremely trivial. All we have to do is sort our movies based on ratings and popularity and display the top movies of our list. As an added step, we can pass in a genre argument to get the top movies of a particular genre. 

In [34]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import warnings; warnings.simplefilter('ignore')

In [83]:
md = pd. read_csv('../data/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [36]:
md.columns

Index([u'adult', u'belongs_to_collection', u'budget', u'genres', u'homepage',
       u'id', u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date', u'revenue', u'runtime',
       u'spoken_languages', u'status', u'tagline', u'title', u'video',
       u'vote_average', u'vote_count'],
      dtype='object')

In [87]:
md = (md.drop(['budget'], axis=1)
         .join(md['budget'].apply(pd.to_numeric, errors='coerce')))

md = md[md['budget'].notnull()]

In [90]:


md = md.replace(0,'NaN')
md = md[md.budget.notnull()]
md = md[md.budget != 'NaN']
md.loc[:, 'budget'] = pd.to_numeric(md['budget'])
md = md[md['budget'] != 0]


In [91]:
md.shape[0]

5381

In [92]:
md = md[md['production_companies'] != '[]']
md.shape[0]

5200

In [53]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0


In [93]:
import json

movieSet = set()

for i in range(0,len(md['production_companies'])):
    print i
    try:
        movieSet.add(md['production_companies'][i][11:md['production_companies'][i].index(",")-1])
    except:
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851


In [94]:
len(movieSet)

415

In [42]:
md['production_companies'][1][11:md['production_companies'][0].index(",")-1]

'Pixar Animation Studios'

In [41]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

I use the TMDB Ratings to come up with our **Top Movies Chart.** I will use IMDB's *weighted rating* formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

The next step is to determine an appropriate value for *m*, the minimum votes required to be listed in the chart. We will use **95th percentile** as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

I will build our overall Top 250 Chart and will define a function to build charts for a particular genre. Let's begin!

In [5]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [6]:
m = vote_counts.quantile(0.95)
m

434.0

In [7]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [8]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres','homepage','poster_path']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 8)

In [9]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [11]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [12]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,homepage,poster_path,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",http://inceptionmovie.warnerbros.com/,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",http://thedarkknight.warnerbros.com/dvdsite/,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",http://www.interstellarmovie.net/,/nBNZadXqJSdt05SHLqgT0HuC5Gm.jpg,7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],http://www.foxmovies.com/movies/fight-club,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",http://www.lordoftherings.net/,/bxVxZb5O9OxCO0oRUNdCnpy9NST.jpg,7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",NaN,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",NaN,/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg,7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",http://www.lordoftherings.net,/uexxR7Kw1qYbZk0RYaF9Rx5ykbj.jpg,7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",NaN,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",http://www.lordoftherings.net/,/wf3v0Pn09jnT5HSaYal7Ami3bdA.jpg,7.851924


In [199]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [200]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity','homepage','poster_path']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [201]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,homepage,poster_path,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,NaN,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,8.565285
351,Forrest Gump,1994,8147,8,48.3072,NaN,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.971357
876,Vertigo,1958,1162,8,18.2082,NaN,/obhM86qyv8RsE69XSMTtT9FdE0b.jpg,7.811667
40251,Your Name.,2016,1030,8,34.461252,https://www.funimationfilms.com/movie/yourname/,/xq1Ugd62d23K2knRUx6xxuALTZB.jpg,7.789489
883,Some Like It Hot,1959,835,8,11.8451,NaN,/pxc9EFCMYkItESpqqrI783yl8Gh.jpg,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,NaN,/xKAweeF2ZPMNn6ce4GclSbr59Pv.jpg,7.744878
19901,Paperman,2012,734,8,7.19863,http://www.disneyanimation.com/projects/shorts...,/3TpMBcAYH4cxCw5WoRacWodMTCG.jpg,7.713951
37863,Sing Street,2016,669,8,10.672862,NaN,/s0C78plmx3dFcO3WMnoXCz56FiN.jpg,7.689483
882,The Apartment,1960,498,8,11.9943,NaN,/6OH3HlplEGoGjqeqRmWaVBGqbav.jpg,7.599317
38718,The Handmaiden,2016,453,8,16.727405,NaN,/wvzfK5QR6dGLwND8MCzWjsQWG4Q.jpg,7.566166


In [219]:
genreSet = list(set(s))
dfGenre = build_chart(genreSet[2]).head(5)
dfGenre['Genre'] = genreSet[2]
for i in range(3,len(genreSet)):
    try:
        x = build_chart(genreSet[i]).head(5)
        x['Genre'] = genreSet[i]
        dfGenre = dfGenre.append(x)
    except:
        continue

In [220]:
dfGenre

,title,year,vote_count,vote_average,popularity,homepage,poster_path,wr,Genre
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,NaN,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,8.565285,Romance
351,Forrest Gump,1994,8147,8,48.3072,NaN,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.971357,Romance
876,Vertigo,1958,1162,8,18.2082,NaN,/obhM86qyv8RsE69XSMTtT9FdE0b.jpg,7.811667,Romance
40251,Your Name.,2016,1030,8,34.461252,https://www.funimationfilms.com/movie/yourname/,/xq1Ugd62d23K2knRUx6xxuALTZB.jpg,7.789489,Romance
883,Some Like It Hot,1959,835,8,11.8451,NaN,/pxc9EFCMYkItESpqqrI783yl8Gh.jpg,7.745154,Romance
359,The Lion King,1994,5520,8,21.6058,http://movies.disney.com/the-lion-king,/bKPtXn9n4M4s8vvZrbw40mYsefB.jpg,7.909339,Animation
5481,Spirited Away,2001,3968,8,41.0489,http://movies.disney.com/spirited-away,/ynXoOxmDHNQ4UAy0oU6avW71HVW.jpg,7.875933,Animation
9698,Howl's Moving Castle,2004,2049,8,16.136,NaN,/iMarB2ior30OAXjPa7QIdeyUfM1.jpg,7.772103,Animation
2884,Princess Mononoke,1997,2041,8,17.1667,NaN,/gzlJkVfWV5VEG5xK25cvFGJgkDz.jpg,7.771305,Animation
5833,My Neighbor Totoro,1988,1730,8,13.5073,NaN,/2i0OOjvi7CqNQA6ZtYJtL65P9oZ.jpg,7.735274,Animation


In [222]:
dfGenre.homepage.fillna('https://www.google.com/search?q=' + dfGenre.title.str.replace(' ','+'), inplace=True)

In [223]:
dfGenre

,title,year,vote_count,vote_average,popularity,homepage,poster_path,wr,Genre
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,https://www.google.com/search?q=Dilwale+Dulhan...,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,8.565285,Romance
351,Forrest Gump,1994,8147,8,48.3072,https://www.google.com/search?q=Forrest+Gump,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,7.971357,Romance
876,Vertigo,1958,1162,8,18.2082,https://www.google.com/search?q=Vertigo,/obhM86qyv8RsE69XSMTtT9FdE0b.jpg,7.811667,Romance
40251,Your Name.,2016,1030,8,34.461252,https://www.funimationfilms.com/movie/yourname/,/xq1Ugd62d23K2knRUx6xxuALTZB.jpg,7.789489,Romance
883,Some Like It Hot,1959,835,8,11.8451,https://www.google.com/search?q=Some+Like+It+Hot,/pxc9EFCMYkItESpqqrI783yl8Gh.jpg,7.745154,Romance
359,The Lion King,1994,5520,8,21.6058,http://movies.disney.com/the-lion-king,/bKPtXn9n4M4s8vvZrbw40mYsefB.jpg,7.909339,Animation
5481,Spirited Away,2001,3968,8,41.0489,http://movies.disney.com/spirited-away,/ynXoOxmDHNQ4UAy0oU6avW71HVW.jpg,7.875933,Animation
9698,Howl's Moving Castle,2004,2049,8,16.136,https://www.google.com/search?q=Howl's+Moving+...,/iMarB2ior30OAXjPa7QIdeyUfM1.jpg,7.772103,Animation
2884,Princess Mononoke,1997,2041,8,17.1667,https://www.google.com/search?q=Princess+Mononoke,/gzlJkVfWV5VEG5xK25cvFGJgkDz.jpg,7.771305,Animation
5833,My Neighbor Totoro,1988,1730,8,13.5073,https://www.google.com/search?q=My+Neighbor+To...,/2i0OOjvi7CqNQA6ZtYJtL65P9oZ.jpg,7.735274,Animation


In [225]:
dfGenre.to_csv("../data/top5Genre.csv")

## Content Based Recommender

The recommender we built in the previous section suffers some severe limitations. For one, it gives the same recommendation to everyone, regardless of the user's personal taste. If a person who loves romantic movies (and hates action) were to look at our Top 15 Chart, s/he wouldn't probably like most of the movies. If s/he were to go one step further and look at our charts by genre, s/he wouldn't still be getting the best recommendations.

For instance, consider a person who loves *Dilwale Dulhania Le Jayenge*, *My Name is Khan* and *Kabhi Khushi Kabhi Gham*. One inference we can obtain is that the person loves the actor Shahrukh Khan and the director Karan Johar. Even if s/he were to access the romance chart, s/he wouldn't find these as the top recommendations.

To personalise our recommendations more, I am going to build an engine that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. Since we will be using movie metadata (or content) to build this engine, this also known as **Content Based Filtering.**

I will build two Content Based Recommenders based on:
* Movie Overviews and Taglines
* Movie Cast, Crew, Keywords and Genre

Also, as mentioned in the introduction, I will be using a subset of all the movies available to us due to limiting computing power available to me. 

In [227]:
links_small = pd.read_csv('../data/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [228]:
md = md.drop([19730, 29503, 35587])

In [229]:
#Check EDA Notebook for how and why I got these indices.
md['id'] = md['id'].astype('int')

In [230]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

We have **9099** movies avaiable in our small movies metadata dataset which is 5 times smaller than our original dataset of 45000 movies.

### Movie Description Based Recommender

Let us first try to build a recommender using movie descriptions and taglines. We do not have a quantitative metric to judge our machine's performance so this will have to be done qualitatively.

In [232]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [233]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [234]:
tfidf_matrix.shape

(9099, 268124)

#### Cosine Similarity

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's **linear_kernel** instead of cosine_similarities since it is much faster.

In [236]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [237]:
cosine_sim[0]

array([ 1.        ,  0.00680476,  0.        , ...,  0.        ,
        0.00344913,  0.        ])

We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 30 most similar movies based on the cosine similarity score.

In [239]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [240]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

We're all set. Let us now try and get the top recommendations for a few movies and see how good the recommendations are.

In [242]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, that is all this system can do at the moment. This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. Someone who liked **The Dark Knight** probably likes it more because of Nolan and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, we are going to use much more suggestive metadata than **Overview** and **Tagline**. In the next subsection, we will build a more sophisticated recommender that takes **genre**, **keywords**, **cast** and **crew** into consideration.

### Metadata Based Recommender

To build our standard metadata based content recommender, we will need to merge our current dataset with the crew and the keyword datasets. Let us prepare this data as our first step.

In [243]:
credits = pd.read_csv('../data/credits.csv')
keywords = pd.read_csv('../data/keywords.csv')

In [244]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [245]:
md.shape

(45463, 25)

In [246]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [452]:
smd = md[md['id'].isin(links_small)]
smd = smd[:3000]
smd.shape

(3000, 28)

We now have our cast, crew, genres and credits, all in one dataframe. Let us wrangle this a little more using the following intuitions:

1. **Crew:** From the crew, we will only pick the director as our feature since the others don't contribute that much to the *feel* of the movie.
2. **Cast:** Choosing Cast is a little more tricky. Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list. 

In [453]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [454]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [455]:
smd['director'] = smd['crew'].apply(get_director)

In [456]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [457]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

My approach to building the recommender is going to be extremely *hacky*. What I plan on doing is creating a metadata dump for every movie which consists of **genres, director, main actors and keywords.** I then use a **Count Vectorizer** to create our count matrix as we did in the Description Recommender. The remaining steps are similar to what we did earlier: we calculate the cosine similarities and return movies that are most similar.

These are steps I follow in the preparation of my genres and credits data:
1. **Strip Spaces and Convert to Lowercase** from all our features. This way, our engine will not confuse between **Johnny Depp** and **Johnny Galecki.** 
2. **Mention Director 3 times** to give it more weight relative to the entire cast.

In [458]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [459]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

#### Keywords

We will do a small amount of pre-processing of our keywords before putting them to any use. As a first step, we calculate the frequenct counts of every keyword that appears in the dataset.

In [460]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [461]:
s = s.value_counts()
s[:5]

independent film    230
woman director      172
murder              135
based on novel      116
suspense             94
Name: keyword, dtype: int64

Keywords occur in frequencies ranging from 1 to 610. We do not have any use for keywords that occur only once. Therefore, these can be safely removed. Finally, we will convert every word to its stem so that words such as *Dogs* and *Dog* are considered the same.

In [462]:
s = s[s > 1]

In [463]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

u'dog'

In [464]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [466]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [467]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [468]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [469]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [471]:
get_recommendations('Mad Max').head(10)

2973          Mad Max 2: The Road Warrior
2974           Mad Max Beyond Thunderdome
1911                Babe: Pig in the City
2451                        The Omega Man
522            Terminator 2: Judgment Day
2045    Battle for the Planet of the Apes
983                    Return of the Jedi
2042                          Logan's Run
2885                    Battlefield Earth
31                         Twelve Monkeys
Name: title, dtype: object

In [ ]:
idx = indices[title]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:31]
movie_indices = [i[0] for i in sim_scores]
#return titles.iloc[movie_indices]

In [472]:
titleDF = pd.DataFrame(titles)


In [473]:
cosine_simDF = pd.DataFrame(cosine_sim)

In [395]:
cosine_simDF.to_csv("../data/cosine_sim.csv")

In [474]:
totalDF = pd.concat([titleDF, cosine_simDF],axis =1)

In [475]:
totalDF

,title,0,1,2,3,4,5,6,7,8,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,Toy Story,1.000000,0.037980,0.045326,0.020847,0.019407,0.000000,0.020332,0.025055,0.000000,...,0.025055,0.000000,0.000000,0.018990,0.000000,0.000000,0.000000,0.020332,0.000000,0.000000
1,Jumanji,0.037980,1.000000,0.000000,0.023357,0.000000,0.000000,0.000000,0.056143,0.025392,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045561,0.000000,0.000000
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.055749,0.025950,0.000000,0.054373,0.000000,0.000000,...,0.033501,0.000000,0.028618,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Waiting to Exhale,0.020847,0.023357,0.055749,1.000000,0.047741,0.018016,0.075023,0.030817,0.000000,...,0.030817,0.026325,0.078975,0.000000,0.000000,0.018490,0.025641,0.000000,0.025641,0.021592
4,Father of the Bride Part II,0.019407,0.000000,0.025950,0.047741,1.000000,0.000000,0.046562,0.000000,0.000000,...,0.028689,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046562,0.000000,0.000000
5,Heat,0.000000,0.000000,0.000000,0.018016,0.000000,1.000000,0.000000,0.043305,0.039171,...,0.043305,0.036993,0.018496,0.000000,0.086432,0.077948,0.144127,0.070284,0.072063,0.151707
6,Sabrina,0.020332,0.000000,0.054373,0.075023,0.046562,0.000000,1.000000,0.000000,0.000000,...,0.030056,0.000000,0.025675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Tom and Huck,0.025055,0.056143,0.000000,0.030817,0.000000,0.043305,0.000000,1.000000,0.100504,...,0.037037,0.031639,0.031639,0.000000,0.024641,0.022222,0.030817,0.060111,0.030817,0.025950
8,Sudden Death,0.000000,0.025392,0.000000,0.000000,0.000000,0.039171,0.000000,0.100504,1.000000,...,0.067003,0.000000,0.000000,0.000000,0.022288,0.020101,0.027875,0.081559,0.000000,0.023473
9,GoldenEye,0.000000,0.016623,0.000000,0.000000,0.000000,0.025643,0.000000,0.043863,0.059514,...,0.043863,0.000000,0.000000,0.000000,0.014591,0.026318,0.018248,0.088988,0.000000,0.015366


In [476]:
totalDF.to_csv('../data/contentRec_3000.csv', index = False)

In [383]:
totalDF.shape[0]

9219

In [369]:
lastComplete = 0
for i in range(1,10):
    partialDF = totalDF[lastComplete:int(totalDF.shape[0]*(float(i)/9))]
    lastComplete = int(totalDF.shape[0]*(float(i)/9))
    partialDF.to_csv('../data/contentRec'+str(i) +'.csv')


In [322]:
partialDF =totalDF[lastComplete:int(totalDF.shape[0]*(float(1)/10))]

In [323]:
partialDF

,title,0,1,2,3,4,5,6,7,8,...,9209,9210,9211,9212,9213,9214,9215,9216,9217,9218
0,Toy Story,1.000000,0.037980,0.045326,0.019854,0.018990,0.000000,0.020332,0.025055,0.000000,...,0.000000,0.000000,0.018990,0.000000,0.016669,0.026038,0.000000,0.000000,0.000000,0.000000
1,Jumanji,0.037980,1.000000,0.000000,0.022244,0.000000,0.000000,0.000000,0.056143,0.023980,...,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,0.000000,0.027086,0.044488,0.000000
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.053093,0.025392,0.000000,0.054373,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.022288,0.000000,0.033501,0.032325,0.000000,0.000000
3,Waiting to Exhale,0.019854,0.022244,0.053093,1.000000,0.044488,0.015813,0.071449,0.029348,0.000000,...,0.000000,0.017379,0.044488,0.030500,0.019525,0.030500,0.029348,0.056637,0.023256,0.000000
4,Father of the Bride Part II,0.018990,0.000000,0.025392,0.044488,1.000000,0.000000,0.045561,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.018676,0.000000,0.000000,0.000000,0.000000,0.000000
5,Heat,0.000000,0.000000,0.000000,0.015813,0.000000,1.000000,0.000000,0.039912,0.034095,...,0.000000,0.106354,0.045376,0.062217,0.000000,0.020739,0.019956,0.019256,0.031627,0.000000
6,Sabrina,0.020332,0.000000,0.054373,0.071449,0.045561,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.035595,0.000000,0.000000,0.019996,0.000000,0.030056,0.029001,0.000000,0.000000
7,Tom and Huck,0.025055,0.056143,0.000000,0.029348,0.000000,0.039912,0.000000,1.000000,0.094916,...,0.000000,0.021932,0.112287,0.038490,0.000000,0.038490,0.000000,0.107211,0.146742,0.000000
8,Sudden Death,0.000000,0.023980,0.000000,0.000000,0.000000,0.034095,0.000000,0.094916,1.000000,...,0.000000,0.018735,0.047960,0.032880,0.000000,0.000000,0.031639,0.030528,0.075212,0.000000
9,GoldenEye,0.000000,0.016011,0.000000,0.000000,0.000000,0.022764,0.000000,0.042248,0.054135,...,0.000000,0.037526,0.032022,0.021953,0.000000,0.000000,0.021124,0.020383,0.033478,0.000000


In [370]:
fullDF = pd.read_csv('../data/contentRec1.csv', index_col = 0)
for i in range(2,10):

    fullDF = fullDF.append(pd.read_csv('../data/contentRec'+str(i) +'.csv'))

In [349]:
fullDF.shape

(9219, 9221)

In [350]:
indices = pd.Series(fullDF.index, index=fullDF['title'])
titles = fullDF['title']
fullDF = fullDF.drop(['title','Unnamed: 0'],axis = 1)

print indices


In [357]:
fullDF.values

array([[ 1.        ,  0.03798001,  0.04066418, ...,  0.        ,
         0.        ,  0.02084691],
       [ 0.03798001,  1.        ,  0.02278028, ...,  0.01899   ,
         0.        ,  0.        ],
       [ 0.04532596,  0.        ,  0.05437272, ...,  0.02266298,
         0.        ,  0.05574947],
       ..., 
       [ 0.        ,  0.02708645,  0.05800148, ...,  0.0967019 ,
         0.06611395,  0.0594701 ],
       [ 0.        ,  0.04448841,  0.02381628, ...,  0.03970725,
         0.01809825,  0.02441931],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.04407596,  0.        ]])

In [384]:
def get_recommendations_manual(title):
    totalDF = totalDF
    indices = pd.Series(totalDF.index, index=totalDF['title'])
    titles = totalDF['title']
    totalDF = totalDF.drop(['title','Unnamed: 0'],axis = 1)    
    idx = indices[title]
    cosine_sim = total.values
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [385]:
get_recommendations_manual("The Dark Knight")

UnboundLocalError: local variable 'totalDF' referenced before assignment

NameError: name 'total' is not defined

In [448]:
totalDF = pd.read_csv('../data/contentRec.csv', memory_map=True)
totalDF = totalDF[:3000]
indices = pd.Series(totalDF.index, index=totalDF['title'])
titles = totalDF['title']
totalDF = totalDF.drop(['title'],axis = 1)    


In [449]:
idx = indices['Mad Max']

cosine_sim = totalDF.values
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:5]
movie_indices = [i[0] for i in sim_scores]
titles.iloc[movie_indices].tolist()

IndexError: positional indexers are out-of-bounds

In [450]:
movie_indices

[2974, 2973, 8864, 6905]

In [427]:
list(enumerate(cosine_sim))[6218]

array([ 0.01183536,  0.01326045,  0.        , ...,  0.01688139,
        0.02772701,  0.        ])

In [431]:
cosine_sim[idx]

array([ 0.        ,  0.01546166,  0.        , ...,  0.01968367,
        0.03232963,  0.        ])

In [584]:
totalDF = pd.read_csv('../data/contentRec_3000.csv', memory_map=True)

In [442]:
totalDF[:3000]

,title,0,1,2,3,4,5,6,7,8,...,9209,9210,9211,9212,9213,9214,9215,9216,9217,9218
0,Toy Story,1.000000,0.037980,0.045326,0.019854,0.018990,0.000000,0.020332,0.025055,0.000000,...,0.000000,0.000000,0.018990,0.000000,0.016669,0.026038,0.000000,0.000000,0.000000,0.000000
1,Jumanji,0.037980,1.000000,0.000000,0.022244,0.000000,0.000000,0.000000,0.056143,0.023980,...,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,0.000000,0.027086,0.044488,0.000000
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.053093,0.025392,0.000000,0.054373,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.022288,0.000000,0.033501,0.032325,0.000000,0.000000
3,Waiting to Exhale,0.019854,0.022244,0.053093,1.000000,0.044488,0.015813,0.071449,0.029348,0.000000,...,0.000000,0.017379,0.044488,0.030500,0.019525,0.030500,0.029348,0.056637,0.023256,0.000000
4,Father of the Bride Part II,0.018990,0.000000,0.025392,0.044488,1.000000,0.000000,0.045561,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.018676,0.000000,0.000000,0.000000,0.000000,0.000000
5,Heat,0.000000,0.000000,0.000000,0.015813,0.000000,1.000000,0.000000,0.039912,0.034095,...,0.000000,0.106354,0.045376,0.062217,0.000000,0.020739,0.019956,0.019256,0.031627,0.000000
6,Sabrina,0.020332,0.000000,0.054373,0.071449,0.045561,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.035595,0.000000,0.000000,0.019996,0.000000,0.030056,0.029001,0.000000,0.000000
7,Tom and Huck,0.025055,0.056143,0.000000,0.029348,0.000000,0.039912,0.000000,1.000000,0.094916,...,0.000000,0.021932,0.112287,0.038490,0.000000,0.038490,0.000000,0.107211,0.146742,0.000000
8,Sudden Death,0.000000,0.023980,0.000000,0.000000,0.000000,0.034095,0.000000,0.094916,1.000000,...,0.000000,0.018735,0.047960,0.032880,0.000000,0.000000,0.031639,0.030528,0.075212,0.000000
9,GoldenEye,0.000000,0.016011,0.000000,0.000000,0.000000,0.022764,0.000000,0.042248,0.054135,...,0.000000,0.037526,0.032022,0.021953,0.000000,0.000000,0.021124,0.020383,0.033478,0.000000


In [516]:
totalDF

,title,0,1,2,3,4,5,6,7,8,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,Toy Story,1.000000,0.037980,0.045326,0.020847,0.019407,0.000000,0.020332,0.025055,0.000000,...,0.025055,0.000000,0.000000,0.018990,0.000000,0.000000,0.000000,0.020332,0.000000,0.000000
1,Jumanji,0.037980,1.000000,0.000000,0.023357,0.000000,0.000000,0.000000,0.056143,0.025392,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045561,0.000000,0.000000
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.055749,0.025950,0.000000,0.054373,0.000000,0.000000,...,0.033501,0.000000,0.028618,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Waiting to Exhale,0.020847,0.023357,0.055749,1.000000,0.047741,0.018016,0.075023,0.030817,0.000000,...,0.030817,0.026325,0.078975,0.000000,0.000000,0.018490,0.025641,0.000000,0.025641,0.021592
4,Father of the Bride Part II,0.019407,0.000000,0.025950,0.047741,1.000000,0.000000,0.046562,0.000000,0.000000,...,0.028689,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046562,0.000000,0.000000
5,Heat,0.000000,0.000000,0.000000,0.018016,0.000000,1.000000,0.000000,0.043305,0.039171,...,0.043305,0.036993,0.018496,0.000000,0.086432,0.077948,0.144127,0.070284,0.072063,0.151707
6,Sabrina,0.020332,0.000000,0.054373,0.075023,0.046562,0.000000,1.000000,0.000000,0.000000,...,0.030056,0.000000,0.025675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Tom and Huck,0.025055,0.056143,0.000000,0.030817,0.000000,0.043305,0.000000,1.000000,0.100504,...,0.037037,0.031639,0.031639,0.000000,0.024641,0.022222,0.030817,0.060111,0.030817,0.025950
8,Sudden Death,0.000000,0.025392,0.000000,0.000000,0.000000,0.039171,0.000000,0.100504,1.000000,...,0.067003,0.000000,0.000000,0.000000,0.022288,0.020101,0.027875,0.081559,0.000000,0.023473
9,GoldenEye,0.000000,0.016623,0.000000,0.000000,0.000000,0.025643,0.000000,0.043863,0.059514,...,0.043863,0.000000,0.000000,0.000000,0.014591,0.026318,0.018248,0.088988,0.000000,0.015366


In [585]:
metaData = pd.read_csv('../data/movies_metadata.csv')

In [529]:
metaList = metaData[metaData["title"].isin(totalDF['title'].tolist())]

In [587]:
titleTemp = pd.DataFrame(totalDF['title'])

In [588]:
titleTemp

,title
0,Toy Story
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II
5,Heat
6,Sabrina
7,Tom and Huck
8,Sudden Death
9,GoldenEye


In [592]:
titleTemp.merge(metaList, on='title',how = 'right').drop_duplicates(['title'])

,title,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,video,vote_average,vote_count
0,Toy Story,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,False,7.7,5415.0
1,Jumanji,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,False,6.9,2413.0
2,Grumpier Old Men,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,False,6.5,92.0
3,Waiting to Exhale,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,False,6.1,34.0
4,Father of the Bride Part II,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,False,5.7,173.0
5,Heat,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,False,7.7,1886.0
8,Sabrina,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,...,"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'FranÃ§ais'}, {'i...",Released,You are cordially invited to the most surprisi...,False,6.2,141.0
12,Tom and Huck,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,False,5.4,45.0
13,Sudden Death,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,False,5.5,174.0
14,GoldenEye,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,...,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,False,6.6,1194.0


In [500]:
for index, row in metaList.iterrows():
    totalDF['homepage'][index] = metaList['homepage'][index]
    totalDF['poster_path'][index] = metaList['poster_path'][index]
    totalDF['release_date'][index] = metaList['release_date'][index]

KeyboardInterrupt: 

In [499]:
metaList.columns

Index([u'adult', u'belongs_to_collection', u'budget', u'genres', u'homepage',
       u'id', u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date', u'revenue', u'runtime',
       u'spoken_languages', u'status', u'tagline', u'title', u'video',
       u'vote_average', u'vote_count'],
      dtype='object')

In [565]:
result = totalDF.merge(metaList, on='title',how = 'left')


In [566]:
result[2500:3000]

,title,0,1,2,3,4,5,6,7,8,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,video,vote_average,vote_count
2500,Wilde,0.000000,0.000000,0.000000,0.046714,0.021744,0.016411,0.022780,0.028072,0.000000,...,"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1997-09-01,0.0,118.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Loved for being unique. Hated for being differ...,False,6.7,62.0
2501,Outside Ozona,0.020332,0.000000,0.054373,0.125039,0.023281,0.035142,0.048780,0.030056,0.027186,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1998-12-18,0.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Somewhere down the road is the end of the line.,False,6.0,2.0
2502,Affliction,0.000000,0.000000,0.000000,0.034943,0.000000,0.024551,0.000000,0.041996,0.000000,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1997-08-28,6330054.0,114.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,False,5.8,56.0
2503,Another Day in Paradise,0.000000,0.000000,0.000000,0.027067,0.000000,0.133122,0.000000,0.032530,0.029424,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1998-12-30,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,We live at the abyss.,False,6.4,38.0
2504,The Hi-Lo Country,0.000000,0.000000,0.027186,0.050016,0.000000,0.035142,0.024390,0.060111,0.027186,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1998-12-30,0.0,114.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A woman like Mona can drive men to extremes.,False,4.7,9.0
2505,Hilary and Jackie,0.000000,0.000000,0.000000,0.050016,0.023281,0.017571,0.024390,0.030056,0.000000,...,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1998-12-30,0.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Two sisters. Two lives. One Love...,False,6.5,23.0
2506,Playing by Heart,0.000000,0.000000,0.000000,0.050016,0.023281,0.017571,0.024390,0.030056,0.000000,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1998-12-30,3970078.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If romance is a mystery, there's only one way ...",False,6.6,49.0
2507,At First Sight,0.000000,0.000000,0.034816,0.096077,0.000000,0.045004,0.031235,0.038490,0.000000,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1999-01-15,0.0,128.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Science gave him sight. She gave him vision.,False,5.9,49.0
2508,In Dreams,0.000000,0.021277,0.000000,0.093428,0.000000,0.065644,0.000000,0.028072,0.025392,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1999-01-15,0.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,You don't have to sleep to dream,False,5.4,45.0
2509,Varsity Blues,0.017244,0.019320,0.046114,0.106047,0.019745,0.014902,0.041371,0.025491,0.000000,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1999-01-15,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It takes a hero to know what's worth winning.,False,6.0,126.0


In [567]:
result.columns

Index([u'title', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8',
       ...
       u'production_countries', u'release_date', u'revenue', u'runtime',
       u'spoken_languages', u'status', u'tagline', u'video', u'vote_average',
       u'vote_count'],
      dtype='object', length=3024)

In [568]:
result = result.drop(['revenue','runtime','spoken_languages','status','tagline','video','vote_average','vote_count'], axis = 1)

In [569]:
result.columns

Index([u'title', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8',
       ...
       u'id', u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date'],
      dtype='object', length=3016)

In [570]:
result = result.drop(['imdb_id','original_language','original_title','overview','popularity','production_companies','production_countires'], axis = 1)

ValueError: labels ['production_countires'] not contained in axis

In [571]:
result.columns

Index([u'title', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8',
       ...
       u'id', u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date'],
      dtype='object', length=3016)

In [572]:
result = result.drop(['adult','belongs_to_collection','budget','genres','id','production_countries'],axis = 1)

In [573]:
result.columns

Index([u'title', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8',
       ...
       u'2999', u'homepage', u'imdb_id', u'original_language',
       u'original_title', u'overview', u'popularity', u'poster_path',
       u'production_companies', u'release_date'],
      dtype='object', length=3010)

In [574]:
result.shape

(3767, 3010)

In [575]:
result

,title,0,1,2,3,4,5,6,7,8,...,2999,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,release_date
0,Toy Story,1.000000,0.037980,0.045326,0.020847,0.019407,0.000000,0.020332,0.025055,0.000000,...,0.000000,http://toystory.disney.com/toy-story,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30
1,Jumanji,0.037980,1.000000,0.000000,0.023357,0.000000,0.000000,0.000000,0.056143,0.025392,...,0.000000,NaN,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.055749,0.025950,0.000000,0.054373,0.000000,0.000000,...,0.000000,NaN,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22
3,Waiting to Exhale,0.020847,0.023357,0.055749,1.000000,0.047741,0.018016,0.075023,0.030817,0.000000,...,0.021592,NaN,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22
4,Father of the Bride Part II,0.019407,0.000000,0.025950,0.047741,1.000000,0.000000,0.046562,0.000000,0.000000,...,0.000000,NaN,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10
5,Heat,0.000000,0.000000,0.000000,0.018016,0.000000,1.000000,0.000000,0.043305,0.039171,...,0.151707,NaN,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...",1995-12-15
6,Heat,0.000000,0.000000,0.000000,0.018016,0.000000,1.000000,0.000000,0.043305,0.039171,...,0.151707,NaN,tt0068688,en,Heat,"Former child star Joe Davis (Joe Dallesandro),...",0.466019,/l6xBVdKfjteFnQucRRJoyIRJOkm.jpg,"[{'name': 'Andy Warhol Productions', 'id': 100...",1972-10-06
7,Heat,0.000000,0.000000,0.000000,0.018016,0.000000,1.000000,0.000000,0.043305,0.039171,...,0.151707,NaN,tt0093164,en,Heat,Reynolds plays an ex-soldier-of-fortunish char...,1.35223,/lRicKGyG3kjkfvlCvv5kzGlox35.jpg,"[{'name': 'New Century Productions', 'id': 866...",1986-01-01
8,Sabrina,0.020332,0.000000,0.054373,0.075023,0.046562,0.000000,1.000000,0.000000,0.000000,...,0.000000,NaN,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.67728,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",1995-12-15
9,Sabrina,0.020332,0.000000,0.054373,0.075023,0.046562,0.000000,1.000000,0.000000,0.000000,...,0.000000,NaN,tt0047437,en,Sabrina,Linus and David Larrabee are the two sons of a...,7.35974,/7ITDmatHa2yf5UTzjwaKAvf3Xr6.jpg,"[{'name': 'Paramount Pictures', 'id': 4}]",1954-09-28


In [576]:
result.homepage.fillna('https://www.google.com/search?q=' + result.title.str.replace(' ','+'), inplace=True)

In [577]:
result.head()

,title,0,1,2,3,4,5,6,7,8,...,2999,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,release_date
0,Toy Story,1.000000,0.037980,0.045326,0.020847,0.019407,0.000000,0.020332,0.025055,0.000000,...,0.000000,http://toystory.disney.com/toy-story,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30
1,Jumanji,0.037980,1.000000,0.000000,0.023357,0.000000,0.000000,0.000000,0.056143,0.025392,...,0.000000,https://www.google.com/search?q=Jumanji,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15
2,Grumpier Old Men,0.045326,0.000000,1.000000,0.055749,0.025950,0.000000,0.054373,0.000000,0.000000,...,0.000000,https://www.google.com/search?q=Grumpier+Old+Men,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22
3,Waiting to Exhale,0.020847,0.023357,0.055749,1.000000,0.047741,0.018016,0.075023,0.030817,0.000000,...,0.021592,https://www.google.com/search?q=Waiting+to+Exhale,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22
4,Father of the Bride Part II,0.019407,0.000000,0.025950,0.047741,1.000000,0.000000,0.046562,0.000000,0.000000,...,0.000000,https://www.google.com/search?q=Father+of+the+...,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10


In [580]:
result = result.drop(['imdb_id','original_language','original_title','overview','popularity','production_companies'],axis = 1)

In [581]:
result.columns

Index([u'title', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8',
       ...
       u'2993', u'2994', u'2995', u'2996', u'2997', u'2998', u'2999',
       u'homepage', u'poster_path', u'release_date'],
      dtype='object', length=3004)

In [583]:
result.to_csv("../data/contentRec_3000_meta.csv")